# Usage

* In your working repo, move to the git branch you wish to analyze
* Supply your personal Github token in `.env` under `GITHUB_TOKEN`
* Supply the backend full path under the `BACKEND_PATH` variable
* Supply your PR number (if you've already created the PR) under the `STUDY_PR_NUMBER` variable. Otherwise, leave `None`.
* Supply the service file you want to analyze against under the `STUDY_FILE` variable.
* Supply the label name from the sister-PRs to compare with under the `STUDY_PR_LABEL` variable.

In [11]:
from django.conf import settings
from apps.patterns.analyzer import analyze_functions
import apps.patterns.git as git
import apps.patterns.github as gh

def create_pr_metadata_object(pull_req):
    return {"branch": process_pr_label(pull_req.head.label), "pr_number": pull_req.number, "pr_name": pull_req.title}

def process_pr_label(label):
    return label.split(f"{settings.ORG_NAME}:")[1]

# User supplied parameters
BACKEND_PATH = "/Users/seba.gonzalez/repo/cornershop-backend"
STUDY_PR_NUMBER = 15741 or None

# Constants
STUDY_FILE = "apps/orders/services.py"
FILE_PATH = f"{BACKEND_PATH}/{STUDY_FILE}"
STUDY_PR_LABEL = "orders-service-api"

repo_active_branch = git.get_active_branch_name_from_repo(BACKEND_PATH)
github_repo = gh.get_repo()

# Analyse function metadata from your working branch
context = {"branch": repo_active_branch}
current_branch_metadata = analyze_functions(file_path=FILE_PATH, context=context)

# Analyse function metadata from devel ref
devel_study_content = github_repo.get_contents(STUDY_FILE, ref="devel")
devel_study_content = devel_study_content.content
devel_metadata = analyze_functions(content=devel_study_content, context={})

# Clean study branch metadata from data appearing also in devel
metadata_to_delete = [funct_name for funct_name in current_branch_metadata.keys() if funct_name in devel_metadata]
for key in metadata_to_delete: del current_branch_metadata[key]

## TODO Remove current branch from all prs
catalogdb_pr_query = f"repo:{settings.BACKEND_REPO_SLUG} is:pr state:open label:{STUDY_PR_LABEL}"
prs = gh.search_issues_with_query(query=catalogdb_pr_query)
if STUDY_PR_NUMBER is not None:
    prs = [pr for pr in prs if pr.number != STUDY_PR_NUMBER]
repo = gh.get_repo()

# Collect function metadata from all catalogdb active PRs
prs_metadata = []
for pr in prs:
    pull_req = github_repo.get_pull(pr.number)
    pr_branch = process_pr_label(pull_req.head.label)

    # Using the PR files, analyse function metadata for this particular PR
    file_content = github_repo.get_contents(STUDY_FILE, ref=pr_branch)
    context = create_pr_metadata_object(pull_req)
    metadata = analyze_functions(content=file_content.content, context=context)
    
    # Clean PR metadata from data appearing also in devel
    metadata_to_delete = [funct_name for funct_name in metadata.keys() if funct_name in devel_metadata]
    for key in metadata_to_delete: del metadata[key]
        
    prs_metadata.append(metadata)

    
for funct_name, funct_metadata in current_branch_metadata.items():
    found_in = []
    for pr in prs_metadata:
        if funct_name in pr:
            found_in.append(f'{pr[funct_name]["pr_number"]} {pr[funct_name]["pr_name"]}')
    

    funct_metadata["found_in"] = found_in
    current_branch_metadata[funct_name] = funct_metadata

def these_services_are_present_in_someonelse_pr(hits):
    if len(hits) == 0:
        print("No name collision so far!")
        return
    print(f"This services are present in other PRs")
    print("=====================================================")
    for hit, metadata in hits.items():
        ocurrences = metadata["found_in"]
        match_list = [f"    {match}" for match in ocurrences]
        print(f"{hit}")
        print("\n".join(match_list))

def these_are_your_services(hits):
    if len(hits) == 0:
        return
    print(f"These are the services you've created in this branch and don't appear in devel")
    print("=====================================================")
    for hit, metadata in hits.items():
        print(f"{hit}")
        
your_services = {
    k: v 
    for k, v in current_branch_metadata.items()
    if k in list(
        filter(
            lambda x: len(current_branch_metadata[x]["found_in"]) == 0,
            current_branch_metadata
        )
    )
}

present_in_someonelse_pr = {
    k: v 
    for k, v in current_branch_metadata.items()
    if k in list(
        filter(
            lambda x: len(current_branch_metadata[x]["found_in"]) > 0,
            current_branch_metadata
        )
    )
}

these_are_your_services(your_services)
print()
these_services_are_present_in_someonelse_pr(present_in_someonelse_pr)

These are the services you've created in this branch and don't appear in devel
get_last_order_activity_values_by_order_id_and_status
get_order_values_by_order_id

No name collision so far!
